<a href="https://colab.research.google.com/github/freakezoide/codigos-de-coolab/blob/main/Copia_de_dash_interactivo_con_token_ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pyngrok
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.1 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 5s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Configuración de la página
st.set_page_config(
    page_title="Dashboard Educativo",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Título y descripción
st.title("📊 Dashboard de Análisis Educativo")
st.markdown("""
Análisis interactivo del uso de plataformas educativas por estudiantes y docentes.
""")

# Función para cargar datos
@st.cache_data
def load_data():
    # Datos de ejemplo (reemplaza con tus archivos reales)
    try:
        # Ejemplo de datos de estudiantes
        estudiantes_data = {
            'Id persona': [1, 2, 3],
            'Sexo': ['Femenino', 'Masculino', 'Femenino'],
            'Subsistema': ['DGES', 'DGETP', 'DGEIP'],
            'Departamento': ['Montevideo', 'Canelones', 'Maldonado'],
            'Cantidad de días ingreso a CREA': [5, 10, 8],
            'Cantidad de días de ingreso a Biblioteca': [2, 1, 3],
            'Cantidad de días de ingreso a PAM': [1, 2, 1]
        }
        estudiantes_df = pd.DataFrame(estudiantes_data)

        # Ejemplo de datos de docentes
        docentes_data = {
            'Id persona': [101, 102],
            'Sexo': ['Femenino', 'Masculino'],
            'Subsistema': ['DGES', 'DGEIP'],
            'Departamento': ['Montevideo', 'Canelones'],
            'Cantidad de días ingreso a CREA': [15, 12],
            'Cantidad de días de ingreso a Biblioteca': [5, 4],
            'Cantidad de Comentarios posteados': [20, 15]
        }
        docentes_df = pd.DataFrame(docentes_data)

        # Calcular interactividad total
        estudiantes_df['Interactividad Total'] = estudiantes_df.iloc[:, 4:7].sum(axis=1)
        docentes_df['Interactividad Total'] = docentes_df.iloc[:, 4:7].sum(axis=1)

        return estudiantes_df, docentes_df

    except Exception as e:
        st.error(f"Error cargando datos: {e}")
        return pd.DataFrame(), pd.DataFrame()

# Cargar datos
estudiantes_df, docentes_df = load_data()

# Sidebar con controles
st.sidebar.header("Filtros y Configuración")
rol = st.sidebar.radio("Seleccione el rol:", ["Estudiantes", "Docentes", "Comparación"], index=2)

subsistemas = ["Todos"] + list(estudiantes_df['Subsistema'].unique())
subsistema_seleccionado = st.sidebar.selectbox("Subsistema:", subsistemas)

departamentos = ["Todos"] + list(estudiantes_df['Departamento'].unique())
departamento_seleccionado = st.sidebar.selectbox("Departamento:", departamentos)

# Función para aplicar filtros
def aplicar_filtros(df, subsistema, departamento):
    if subsistema != "Todos":
        df = df[df['Subsistema'] == subsistema]
    if departamento != "Todos":
        df = df[df['Departamento'] == departamento]
    return df

# Visualizaciones principales
def mostrar_visualizaciones(est_df, doc_df, rol_seleccionado):
    if rol_seleccionado in ["Estudiantes", "Comparación"]:
        est_filtrado = aplicar_filtros(est_df, subsistema_seleccionado, departamento_seleccionado)
    if rol_seleccionado in ["Docentes", "Comparación"]:
        doc_filtrado = aplicar_filtros(doc_df, subsistema_seleccionado, departamento_seleccionado)

    if rol_seleccionado == "Comparación":
        st.header("Comparación entre Estudiantes y Docentes")

        col1, col2 = st.columns(2)

        with col1:
            st.subheader("Interactividad Total")
            fig = px.box(
                pd.DataFrame({
                    'Rol': ['Estudiantes']*len(est_filtrado) + ['Docentes']*len(doc_filtrado),
                    'Interactividad': list(est_filtrado['Interactividad Total']) + list(doc_filtrado['Interactividad Total'])
                }),
                x='Rol', y='Interactividad',
                title="Distribución de Interactividad Total"
            )
            st.plotly_chart(fig, use_container_width=True)

        with col2:
            st.subheader("Uso de Plataformas (Estudiantes)")
            plataformas_est = ['Cantidad de días ingreso a CREA',
                             'Cantidad de días de ingreso a Biblioteca',
                             'Cantidad de días de ingreso a PAM']

            promedios_est = est_filtrado.groupby('Subsistema')[plataformas_est].mean().reset_index()
            promedios_est_melt = promedios_est.melt(id_vars='Subsistema', var_name='Plataforma', value_name='Promedio')

            fig = px.bar(
                promedios_est_melt,
                x='Subsistema', y='Promedio',
                color='Plataforma',
                barmode='group'
            )
            st.plotly_chart(fig, use_container_width=True)

    elif rol_seleccionado == "Estudiantes":
        st.header("Análisis de Estudiantes")

        col1, col2 = st.columns(2)

        with col1:
            st.subheader("Distribución de Interactividad")
            fig = px.histogram(
                est_filtrado,
                x='Interactividad Total',
                nbins=20
            )
            st.plotly_chart(fig, use_container_width=True)

        with col2:
            st.subheader("Top Departamentos")
            top_deptos = est_filtrado.groupby('Departamento')['Interactividad Total'].mean().nlargest(5).reset_index()
            fig = px.bar(
                top_deptos,
                x='Interactividad Total',
                y='Departamento',
                orientation='h'
            )
            st.plotly_chart(fig, use_container_width=True)

    elif rol_seleccionado == "Docentes":
        st.header("Análisis de Docentes")

        col1, col2 = st.columns(2)

        with col1:
            st.subheader("Distribución de Interactividad")
            fig = px.histogram(
                doc_filtrado,
                x='Interactividad Total',
                nbins=20
            )
            st.plotly_chart(fig, use_container_width=True)

        with col2:
            st.subheader("Top Departamentos")
            top_deptos = doc_filtrado.groupby('Departamento')['Interactividad Total'].mean().nlargest(5).reset_index()
            fig = px.bar(
                top_deptos,
                x='Interactividad Total',
                y='Departamento',
                orientation='h'
            )
            st.plotly_chart(fig, use_container_width=True)

    # Mostrar datos filtrados
    st.subheader("Datos Filtrados")
    if rol_seleccionado == "Estudiantes":
        st.dataframe(est_filtrado)
    elif rol_seleccionado == "Docentes":
        st.dataframe(doc_filtrado)
    else:
        st.write("Estudiantes:")
        st.dataframe(est_filtrado)
        st.write("Docentes:")
        st.dataframe(doc_filtrado)

# Ejecutar visualizaciones
mostrar_visualizaciones(estudiantes_df, docentes_df, rol)

# Información adicional
st.sidebar.markdown("---")
st.sidebar.markdown("""
**Instrucciones:**
1. Seleccione el rol a analizar
2. Filtre por subsistema y/o departamento
3. Explore las visualizaciones
""")

# Estadísticas generales
st.sidebar.markdown("---")
st.sidebar.subheader("Estadísticas Generales")
if not estudiantes_df.empty:
    st.sidebar.write(f"📚 Estudiantes: {len(estudiantes_df)}")
if not docentes_df.empty:
    st.sidebar.write(f"👩🏫 Docentes: {len(docentes_df)}")


In [ ]:
!streamlit run app.py &>/dev/null&
from pyngrok import ngrok

# Configura tu token de ngrok (obtenlo de https://dashboard.ngrok.com/get-started/your-authtoken)
ngrok.set_auth_token("tu_token_aqui")  # Reemplaza con tu token real

public_url = ngrok.connect(addr='8501', proto='http')
print("Tu aplicación está disponible en:", public_url)